# Environment setup
Follow the tutorial about how to utilize Google Colab but **don't install PyTorch** as mentioned in the blog post.

Turkish:
https://medium.com/deep-learning-turkiye/google-colab-ile-%C3%BCcretsiz-gpu-kullan%C4%B1m%C4%B1-30fdb7dd822e

English:
https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d



In [ ]:
# This part probably will be enough for utilizing Drive in Colab
# but examine links above if you encounter with problems.
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

## Now, you will work on Architectural Heritage Elements Dataset and classify these elements into 10 categories

After having mounted the Jupyter Notebook to Google Drive, navigate the following address: https://drive.google.com/drive/folders/1PLXZYjGeaM1rekMKUTo8mlaaPeFOrBft?usp=sharing


Add this folder entirely to your Google Drive. If you have done it correctly, then you should be able to see *data* folder in your drive.

You can examine the dataset in the following address: https://old.datahub.io/dataset/architectural-heritage-elements-image-dataset


### Don't forget to choose the right runtime from the menu above. (GPU should be selected)

In [ ]:
!nvidia-smi
# This command should return some information about the GPU status if the runtime is right. 
# In addition to that, if you encounter memory issues, you can diagnose your model by this command.

### You are free to utilize Pytorch methods in this part of the homework. You will be using pretained models ResNet-50, DenseNet-121 and your own model.

In [ ]:
# All libraries are already presented in Colab Servers, we don't need to install anything with pip
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Pre-trained Models

You can find tutorials on how to load those models at pytorch.org . Don't forget to use pretrained=True if you wish to do finetuning.

In [1]:
# Load ResNet-50 and DenseNet-121 model seperately here

## Your Model

Additionally, build your own model which is different from the other models, train on the Architectural Heritage Elements dataset.

In [ ]:
class YourModel(nn.Module):
    def __init__(self):
        # TO DO: Your neural network design
        super(YourModel, self).__init__()
        pass
        self.seq = nn.Sequential(nn.Linear(10,10))
    def forward(self, x):
        # TO DO: Your neural network design
        out = None
        return out

model = YourModel()

In [ ]:
###  Here are some training parameters which you can tweak
batch_size = 32
learning_rate = 1e-3
regularization_rate = 0
n_epochs = 10
use_gpu = True
test_every = 3
###

# You may want to tweak them too and you can use different parameter settings for different models. 
# These are just examples
optimizer = optim.SGD(params=None, lr=learning_rate)
criteria = nn.CrossEntropyLoss()

## DataLoader

Here we provide you the codes for loading the train data, validation data and test data. Please ensure that you understood how PyTorch methods like ImageFolder, DataLoader and transformations work.

In [ ]:
train_transforms = transforms.Compose([
            transforms.Resize(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
val_transforms = transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
test_transforms = transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

train_dataset = ImageFolder('drive/My Drive/data/train', train_transforms)
val_dataset = ImageFolder('drive/My Drive/data/val', val_transforms)
test_dataset = ImageFolder('drive/My Drive/data/test', test_transforms)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=4, shuffle=True)
test_loader = DataLoader(dataset=val_dataset, batch_size=32, num_workers=4, shuffle=False)
val_loader = DataLoader(dataset=test_dataset, batch_size=32, num_workers=4, shuffle=False)

# Train your own model

Don't forget to include appropriate regularizations. Choose appropriate set of hyperparameters such as Learning Rate etc. You may insert new cells.

In [ ]:
# It modifies the behaviour of modules like BatchNorm and Dropout for training purposes
# Use your own model
model.train()
if use_gpu:
    model.cuda()
    criteria.cuda()

# Some example diagnostics.

# Loss for every iteration
losses_iter_train = []
# Loss for epoch (averaging the iteration-wise loss)
losses_epoch_train = []
accuracy_iter_train = []
accuracy_epoch_train = []

losses_iter_val = []
losses_epoch_val = []
accuracy_iter_val = []
accuracy_epoch_val = []

# Write the training loop
for epoch in range(n_epochs):
    for ix, data in train_loader:
        model.zero_grad()
        img, label = data
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        pass
    if epoch % 3 == 1:
        with torch.no_grad():
            model.eval()
                # Measure the performance in validation set.
            
    model.train()
    
        # Fill the rest...
        


## Test your model

Measure the performance against test set. Complete the code below.

In [ ]:
# It modifies the behaviour of modules like BatchNorm and Dropout for test purposes
# Dropout no longer works when .eval() is called.
# BatchNorm uses the learned parameters

model.eval()

with torch.no_grad():
    for data in test_loader:
        img, label = data
        if use_gpu:
            img = img.cuda()
            label = label.cuda()
        # Fill the rest...

# ResNet-50
## Train ResNet-50

Avoid overfitting and underfitting as much as possible. **Try to get highest validation and test accuracy (at least 65%)**

In [ ]:
# You can use the same training mechanism above. Now, you will use ResNet-50 as your model        

## Testing

In [ ]:
# You can use the same testing mechanism above. Now, you will use the ResNet model you trained above 

### Plot the training and validation losses versus number of iterations or epochs for ResNet-50 on the same plot and obtain test accuracy

# DenseNet-121
## Train DenseNet-121

Avoid overfitting and underfitting as much as possible. **Try to get highest validation and test accuracy (at least 65%)**

In [ ]:
# You can use the same training mechanism above. Now, you will use DenseNet-121 as your model 

## Testing

In [ ]:
# You can use the same testing mechanism above. Now, you will use the DenseNet model you trained above 

### Plot the training and validation losses versus number of iterations or epochs for DenseNet-121 on the same plot and obtain test accuracy

# BatchNorm Comparison

**Create two models one with batchnorm layers and one without batchnorm layers. Train them. If your YourModel() satisfies any of these conditions, you can use it for comparison.**

In [ ]:
# Define your model with BatchNorm and train it. Skip this if you use YourModel() for this condition.

In [ ]:
# Define your model without BatchNorm and train it. Skip this if you use YourModel() for this condition.

**For each, plot the training and validation losses versus number of iterations or epochs and compare test accuracies.**

# Dropout Comparison

**Create two models one with dropout layers and one without dropout layers. Train them. If your YourModel() satisfies any of these conditions, you can use it for comparison.**

In [ ]:
# Define your model with Dropout and train it. Skip this if you use YourModel() for this condition.

In [ ]:
# Define your model without Dropout and train it. Skip this if you use YourModel() for this condition.

**For each, plot the training and validation losses versus number of iterations or epochs and compare test accuracies.**

# Optimizer Comparison

**Optimize two identical models one with SGD+Momentum and one with Adam. If your training for YourModel() satisfies any of these conditions, you can use it for comparison.**

In [ ]:
# Train your model with SGD+Momentum. Skip this if you use YourModel() for this condition.

In [ ]:
# Train your model with Adam. Skip this if you use YourModel() for this condition.

**For each, plot the training and validation losses versus number of iterations or epochs and compare test accuracies**

#### After you have completed the training, save your best model using the following command
#### Upload your best model to Google Drive and copy your link here: *link*

In [ ]:
student_id = 111
torch.save(model.state_dict(), 'drive/blg561/{}.pth'.format(student_id))